In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [2]:
#url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"

df_london = pd.read_csv("international-visitors-london-raw.csv", encoding= 'unicode_escape')

In [3]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [4]:
def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
output = widgets.Output()
plot_output = widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year), description = 'Year:')
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.purpose), description = 'Purpose:')
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description = 'Number:')


def common_filtering(year, purpose, num):
    output.clear_output()
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num),subset=['Visits (000s)','Spend (£m)', 'Nights (000s)']))
    with plot_output:
        sns.kdeplot(common_filter['Visits (000s)'], shade=True)
        plt.show()

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,
                     bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, 
                     bounded_num.value)
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)


dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

In [7]:
input_widgets = widgets.HBox(
[dropdown_year, dropdown_purpose, bounded_num])

tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')


In [8]:
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)